임수진, 디지털 경영 트랙, 석사 1학기

참고자료
- 책 : 이것이 데이터 분석이다
- 블로그 :

# 인터파크 "관심집중 핫이슈" 크롤링

1. 내용
- 인터파크 페이지에서 뮤지컬, 콘서트, 연극 탭의 "관심집중 핫이슈" 작품만을 크롤링
- 인기가 많았던 작품뿐만 아니라 뜨고 있는 작품들을 한꺼번에 확인할 수 있는 페이지임

2. 데이터 상세 정보
- 크롤링 url
    - 뮤지컬 : http://ticket.interpark.com/MusicalIndex.asp
- 피처 설명

In [449]:
# -*- coding: utf-8 -*-

%matplotlib inline

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [479]:
musical_url = "http://ticket.interpark.com/MusicalIndex.asp"

req = requests.get(musical_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
musical_posters_list = soup.find_all(name="dt", attrs={"class":"issue_obj"})

musical_page_urls = []
for index in range(0, len(musical_posters_list)):
    a_url = musical_posters_list[index].find_all('a')
    href_url = a_url[0].get('href')
    musical_page_urls.append(href_url)

# 보여지는 작품 수가 계속 변경됨
count_musical_page_urls = len(musical_page_urls)
musical_page_urls

['http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002018',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=P0002351',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001195',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002842',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20003125',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19019216',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001382',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002020',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002972',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20000762',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19009138',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=16016299',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19006228',

In [480]:
count_musical_page_urls

24

In [ ]:
columns = ['title', 'category', 'location', 'date', 'actor']
df = pd.DataFrame(columns = columns)

for musical_page_url in musical_page_urls[:24]:
    req = requests.get(musical_page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        title_category = soup.find(name="div", attrs={"class":"dt_Name"})
        title = title_category.find(name="span")   
        category = title_category.find(name="dd")
    except AttributeError:
        print("error")
    
#     title_category = soup.find(name="div", attrs={"class":"dt_Name"})
#     title = title_category.find(name="span")
#     category = title_category.find(name="dd")

    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""

    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
        
    basic_info = soup.find(name="ul", attrs={"class":"info_Lst"})
    basic_info = basic_info.text.strip()
    row_basic_info = list(basic_info.split('\n'))
    
    remove_word = ['', '관람시간 보기', '출연', '더보기']
    row_basic_info_final = []

    for item in row_basic_info:
        if item not in remove_word:
            row_basic_info_final.append(item)

    row_location = row_basic_info_final[0]
    if row_location[:2] == '장소':
        row_location = row_location[2:]

    row_date = row_basic_info_final[1]
    if row_date[:2] == '기간':
        row_date = row_date[2:]

    try:
        row_actor = row_basic_info_final[2]

    except IndexError:
        row_actor = ""
        
    row = [row_title, row_category, row_location, row_date, row_actor]
    series = pd.Series(row, index = df.columns)
    df = df.append(series, ignore_index=True)

df

In [476]:
sonia_list = ['a', 'b', 'c', 'd', 'e', 'f']

for i in sonia_list:
    if i == 'c':
        print("pass")
    else:
        print(i)

a
b
pass
d
e
f


In [472]:
for index, value in enumerate(sonia_list):
    print(index+1, value)

1 a
2 b
3 c
4 d
5 e
6 f


In [475]:
# 0, 5
req = requests.get(musical_page_urls[22])
html = req.content
soup = BeautifulSoup(html, 'lxml')

try:
    title_category = soup.find(name="div", attrs={"class":"dt_Name"})
    title = title_category.find(name="span")   
    category = title_category.find(name="dd")
except AttributeError:
    print("error")
    


error


In [461]:

if title is not None:
    row_title = title.text.replace("\n", " ")
else:
    row_title = ""

if category is not None:
    row_category = category.text.replace("\n", " ")
else:
    row_category = ""

basic_info = soup.find(name="ul", attrs={"class":"info_Lst"})
basic_info = basic_info.text.strip()
row_basic_info = list(basic_info.split('\n'))
row_basic_info

AttributeError: 'NoneType' object has no attribute 'find'

In [392]:
remove_word = ['', '관람시간 보기', '출연', '더보기']
row_basic_info_final = []

for item in row_basic_info:
    if item not in remove_word:
        row_basic_info_final.append(item)

row_location = row_basic_info_final[0]
if row_location[:2] == '장소':
    row_location = row_location[2:]
    
row_date = row_basic_info_final[1]
if row_date[:2] == '기간':
    row_date = row_date[2:]

try:
    row_actor = row_basic_info_final[2]
    
except IndexError:
    row_actor = ""

print(row_location)
print(row_date)
print(row_actor)

올림픽공원 우리금융아트홀
2020.03.31 ~ 2020.05.10



In [390]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ['title', 'category', 'location', 'date', 'actor']
df = pd.DataFrame(columns = columns)

# 모든 정보를 하나의 데이터 프레임에 저장합니다.
row = [row_title, row_category, row_location, row_date, row_actor]
series = pd.Series(row, index = df.columns)
df = df.append(series, ignore_index=True)

df

,title,category,location,date,actor
0,트롯연가,뮤지컬 | 120분 (인터미션 : 20분) | 만 7세이상,올림픽공원 우리금융아트홀,2020.03.31 ~ 2020.05.10,


In [207]:
result = soup.find('div', {'class':'Mid_Tab'}).findChildren('span')
result

[<span>New</span>,
 <span id="spReviewCnt"></span>,
 <span id="spExpertCnt"></span>,
 <span id="spQnACnt"></span>]

In [ ]:
result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[1].getText()
int(result.replace(',', ''))